In [2]:
import numpy as np
import pandas as pd 

In [4]:
tab1 = pd.read_csv("./data/table1.csv")
tab1.rename(columns={
    "شناسه‌ی عضو" : 'personID',
    "شناسه‌ی بخش" : 'unitID',
    "ضریب اهمیت در بخش":"importance_unit",
    "میزان انتظار":"expected"
},inplace=True)
tab1.head()

,personID,unitID,importance_unit,expected
0,810,1,0.82,21
1,284,1,0.73,34
2,175,1,0.83,43
3,436,1,0.85,44
4,289,1,0.71,31


In [5]:
tab1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   personID         1484 non-null   int64  
 1   unitID           1484 non-null   int64  
 2   importance_unit  1484 non-null   float64
 3   expected         1484 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 46.5 KB


In [7]:
tab2  = pd.read_csv("./data/table2.csv")
tab2.rename(columns={
  'شناسه‌ی بخش': 'unitID',
  'حقوق ساعتی':'hourly_income'
},inplace=True)
tab2.head()

,unitID,hourly_income
0,1,36000
1,2,26000
2,3,20000
3,4,24000
4,5,35000


In [8]:
tab2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   unitID         10 non-null     int64
 1   hourly_income  10 non-null     int64
dtypes: int64(2)
memory usage: 288.0 bytes


In [9]:
tab3 = pd.read_csv("./data/table3.csv")
tab3.rename(columns={
    'شناسه‌ی عضو':"personID",
    'شناسه‌ی بخش':'unitID',
    'ماه':'month',
    'ساعت کاری':'working_time'
},inplace=True)
tab3.head()

,personID,unitID,month,working_time
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14


In [10]:
tab3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17808 entries, 0 to 17807
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   personID      17808 non-null  int64
 1   unitID        17808 non-null  int64
 2   month         17808 non-null  int64
 3   working_time  17808 non-null  int64
dtypes: int64(4)
memory usage: 556.6 KB
